<a href="https://colab.research.google.com/github/aselshall/AR_HAB/blob/main/RedtideBlooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### KB data

In [30]:
import numpy as np
import pandas as pd
#pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

from pathlib import Path
import random

import gdown
from pathlib import Path
Path('Input').mkdir(parents=True, exist_ok=True)
Path('Output').mkdir(parents=True, exist_ok=True)
file='KB_Data.csv'
file_ID='1Sni_9gbTA6-4sE4PBTkKqtelsH0EMpDC'
url='https://drive.google.com/uc?id='
print('Input files download')
source=url+file_ID
destination='Input/{}'.format(file)
print('from {} to {}'.format(source,destination))
gdown.download(source, destination, quiet=True)




Input files download
from https://drive.google.com/uc?id=1Sni_9gbTA6-4sE4PBTkKqtelsH0EMpDC to Input/KB_Data.csv


'Input/KB_Data.csv'

### [1] KB data for the study area from 1950 to 2020

In [2]:
def KBData(start_year,end_of_year,Zone):
  #[1] Read data
  df=pd.read_csv('Input/KB_Data.csv')

  #[2] Convert date column to a datetime, and set and rename index
  df['Sample Date'] = pd.to_datetime(df['Sample Date'])
  df.index=df['Sample Date']
  df.index.names = ['date']

  #[3] Filter study area
  if Zone==1:
    LL2C=[26.5,28.0,-81.61,-85.0]   #N,S,W,E
  elif Zone==2:
    LL2C=[28.96,30.67,-82.57,-85.45]
  elif Zone==3:
    LL2C=[28.96,30.67,-85.45,-87.66]

  df = df[~(df['Latitude'] <LL2C[0])] 
  df = df[~(df['Latitude'] >LL2C[1])] 
  df = df[~(df['Longitude'] >LL2C[2])] 
  df = df[~(df['Longitude'] <LL2C[3])] 

  #[4] Filter time period of interest 
  df=df['{}-1-1'.format(start_year):'{}-12-31'.format(end_of_year)]



  #[5] Filter variables, and rename variables of interest 
  df = df[['Sample Date','Karenia brevis abundance (cells/L)']]
  df.columns = ['Sample Date','max_cells/L_raw'] 

  #[6] Get number of samples and maximum cell/L per day
  samples_count=df['max_cells/L_raw'].groupby(level=0).count()
  df=df.groupby(level=0).max()
  df['num_of_samples']=samples_count

  #[7] Resample to daily data with the maximum cell counts per day
  df=df.resample('D').max()

  #[8] Save filtered data
  df.to_csv('Output/KB_daily_raw_{}_{}_Zone{}.csv'.format(start_year,end_of_year,Zone))
  #df['{}-1'.format(start_year)].head(5)
  #df.head(5)


In [3]:
def plot_KB(data_plot_flag,df,Q,min_bloom_length,max_gaps_days,n_large_blooms,n_large_bloom_periods,bloom_periods_ratio,start_year,end_of_year,Zone):

    #[1] Plot bloom perios 
    #Full data set
    if data_plot_flag==1:
        mask1=df['max_cells/L_raw']>=1e5
        x1=df.index.copy()[mask1]
        y1=df['max_cells/L_raw'].copy()[mask1]
    elif data_plot_flag==2:   
        mask1=Q['max_cells/L_raw']>=1e5
        x1=Q.index.to_timestamp().copy()[mask1]
        y1=Q['max_cells/L_raw'].copy()[mask1]

    #Periods with large blooms
    mask2=Q['n_large_blooms'].notnull()
    x2=Q.index.to_timestamp().copy()[mask2]
    y2=Q['max_cells/L_raw'].copy()[mask2]
    #display(y2)

    #Bar chart 
    fig=plt.figure(figsize=(15,5))
    ax=fig.add_subplot(1,1,1)
    if Period=='Q':
        BarW=92
    elif Period=='2Q':
        BarW=181

    if data_plot_flag==1:
        L1='Period with large-bloom(s) (Length {} - Gaps {})'.format(min_bloom_length,max_gaps_days)
        L2='Daily data with max. K. brevis abundnce above 1e5 cells/L'                                            
        ax.bar(x2,y2,width=BarW,facecolor='red', align='edge',alpha=0.7, label=L1)
        ax.bar(x1,y1,width=0.9,facecolor='green',align='edge', alpha=0.7, label=L2)
    elif data_plot_flag==2:  
        L1='Period with max. K. brevis abundnce above 1e5 cells/L'
        L2='Period with large-bloom(s) (Length {} - Gaps {})'.format(min_bloom_length,max_gaps_days)
        ax.bar(x1,y1,width=BarW,facecolor='green', align='edge', alpha=0.7, label=L1)
        ax.bar(x2,y2,width=BarW,facecolor='red', align='edge', alpha=0.7, label=L2)
    plt.yscale('log')

    #format the x-ticks and labels
    years = mdates.YearLocator(5)   # every year
    if Period=='Q':
        months = mdates.MonthLocator(bymonth=[1,4,7,10,13])  # every month
    elif Period=='2Q':
        months = mdates.MonthLocator(bymonth=[1,7,13])  # every month
    years_fmt = mdates.DateFormatter('%Y')
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_major_formatter(years_fmt)
    ax.xaxis.set_minor_locator(months)

    #x-axis limit
    #Start=1993
    Start=start_year
    End=end_of_year+1
    start = datetime(year=Start, month=1, day=1, hour=0)
    end   = datetime(year=End, month=1, day=1, hour=0)
    ax.set_xlim(start,end)

    #Axis labels
    #ax.set_xlabel('Periods')
    ax.set_ylabel('Maximum Karenia brevis abundnce (cells/L)')

    #Legend
    ax.legend(loc='best')
    
    #Grid
    #ax.grid(which='major', axis='x')

    #Title
#     title='For a minium bloom length of {} days with a maximum of {}-day gaps (i.e, no data or values below 1e5), \n \
#     the number of large-blooms, number of periods-with-large-bloom(s), ratio of periods-with-large-bloom(s), \
# are {:0.0f}, {} and {:0.2f}, respectively'. \
#     format(min_bloom_length,max_gaps_days,n_large_blooms,n_large_bloom_periods,bloom_periods_ratio)
    if Zone==1:
      Area='Tampa'
    elif Zone==2:
      Area='Apalachee Bay'
    elif Zone==3:
      Area='Panhandle'
    title='Large blooms in {} area'.format(Area)
    ax.set_title(title)

    #Display results
    #plt.show()
    #display(Q)
    
    plt.savefig('Output/KB_{}_{}_{}.jpg'.format(start_year,end_of_year,Area))

In [28]:
def ProcessData(Plot_KB_Flag,Period,start_year,end_of_year,Zone):

  #[0] General
  KBData(start_year,end_of_year,Zone)
  Interpretation=2
  rule2=0.2             #20%
  min_bloom_length=[10]   #bloom larger than or equal: 10,14, 15
  max_gaps_days=[5]       #bloom larger than or equal: 10,14, 15

  #Options and flag
  data_plot_flag=1    #Raw data plot: [1] daily, [2] period 
  Start_KB_df_flag=1  #Start new data frame for saving results
  Save_zos_res_Flag=1  #[1] Save resutls
  NewSeg=0

  for Period in [Period]:
      print('Analysis from {} to {} with rule2 {} and bloom lenght {} day and max gaps {} days for period {} in Zone {}'.\
            format(start_year,end_of_year,rule2,min_bloom_length,max_gaps_days,Period,Zone))

      #Part 1: Karenia brevis bloom analysis
      for min_bloom_length in min_bloom_length:              #If bloom larger than or equal: 10,14, 15
          #print('min_bloom_length:',min_bloom_length)  
          for  max_gaps_days in max_gaps_days:              #If bloom larger than or equal: 10,14, 15

              #[1] Read data, set index, and select columns of interest 
              df=pd.read_csv('Output/KB_daily_raw_{}_{}_Zone{}.csv'.format(start_year,end_of_year,Zone))
              df['date'] = pd.to_datetime(df['date'])
              df['Sample Date'] = pd.to_datetime(df['Sample Date'])
              df['idx']=np.arange(len(df))
              df.set_index('date', inplace=True)
              df=df[['idx','Sample Date','num_of_samples','max_cells/L_raw']]

              #[2] Create new dataframe with samples with potential large_bloom
              df['bloom1e5']=df.loc[(df['max_cells/L_raw']>=1e5),'max_cells/L_raw']   #iloc 4

              #[3] Gap between two samples when large bloom is encountered
              bloom_samples=df['bloom1e5'].notnull()
              df.loc[bloom_samples,'gap_days']=df.loc[bloom_samples,'Sample Date'].diff().dt.days-1  #iloc 5

              #[4] Count new blooms with 5 days and 20% rules
              bl=df[bloom_samples].copy()
              bloom=1
              first_bloom=1

              bl['bloom_lenght_inc']=''     #iloc 6
              bl['20%_condition']=''    #iloc 7
              bl['gap']=''              #iloc 8
              bl['new_bloom']=''         #iloc 9
              bl['new_bloom_idx']=''     #iloc 10
              bl.iloc[0,10]=bloom

              for cont in range(0,len(bl)):
                  #Start a new blooom
                  if first_bloom==1:
                      loc1=0
                      loc2=1
                      idx1=bl.iloc[cont,0]
                      #print('First bloom {} starts at {}'.format(bloom,idx))
                      first_bloom=0
                      sample_count=1
                  else:
                      loc2 +=1

                  #Add bloom number
                  if bl.iloc[cont,5]>max_gaps_days:

                      #bloom length
                      idx2=bl.iloc[cont,0]
                      bl.iloc[cont,6]=idx2-idx1   

                      #20%_condition
                      bl.iloc[cont,7]=bl.iloc[cont,6]*rule2

                      #Gap/Gaps
                      if Interpretation==1:
                          bl.iloc[cont,8]= bl.iloc[cont,5]      
                      elif Interpretation==2:
                          bl.iloc[cont,8]= bl.iloc[loc1:loc2,5].sum() 
                          #print(bl.iloc[cont,8])

                      #Decision: Same bloom
                      if bl.iloc[cont,8] <= bl.iloc[cont,7]:
                          bl.iloc[cont,9]=0

                      #Decision: New bloom
                      elif bl.iloc[cont,8]> bl.iloc[cont,7]:
                          bloom +=1 
                          bl.iloc[cont,9]=1
                          loc1=loc2
                          idx1=bl.iloc[cont,0]

                  #New bloom index 
                  bl.iloc[cont,10]=bloom

              #[5] Remove blooms less than 10 days 
              bloom_number=0
              bl=bl.resample('D').max()

              for new_bloom_idx in range(1,bloom+1):

                  #Filter data for each new bloom
                  temp=bl[bl['new_bloom_idx'].eq(new_bloom_idx)]

                  #Bloom length days
                  idx1=temp.iloc[0,0]
                  idx2=temp.iloc[len(temp)-1,0]
                  bloom_length=idx2-idx1+1
                  if bloom_length>=min_bloom_length:
                      bloom_number+=1

                  #Bloom start and end date
                  date1=temp[temp['idx']==idx1].index.values
                  date2=temp[temp['idx']==idx2].index.values

                  #Record bloom number and length 
                  if bloom_length>=min_bloom_length:
                      mask=(bl.index >= date1[0]) & (bl.index <= date2[0]) 
                      bl.loc[mask,'bloom_length']=bloom_length
                      bl.loc[mask,'bloom_number']=bloom_number
                      #print(new_bloom_idx,':',idx1,idx2,':',bloom_length,':',pd.to_datetime(date1[0]).date(),pd.to_datetime(date2[0]).date())
                  else:
                      bl.loc[date1,'bloom_length']=''

              #[6] Merge dataFrames with no_large bloom and large_bloom and save dataFrame
              df = pd.concat([df, bl], axis=1, sort=False)
              #print(df.columns.values)
              df.columns =['idx', 'Sample Date', 'num_of_samples', 'max_cells/L_raw', 'bloom1e5', \
                      'gap_days', 'idx_b1e5', 'Sample Date', 'num_of_samples_b1e5', 'max_cells/L_raw_b1e5', \
                      'bloom1e5', 'gap_days', 'bloom_lenght_inc', '20%_condition', 'gap', \
                      'new_bloom', 'new_bloom_idx', 'bloom_length', 'bloom_number']

              #[7] Save KB daily output
              df.to_csv('Output/KB_daily_{}_{}_Zone{}.csv'.format(start_year,end_of_year,Zone))

              #[8] Count number of periods with large_bloom and number of bloom per period 

              #Data per period (Q or 2Q)
              df=df[['max_cells/L_raw','max_cells/L_raw_b1e5','gap','bloom_length','bloom_number','num_of_samples','num_of_samples_b1e5']]
              Q=df.resample(Period,kind='period').mean()
              n_periods=len(Q.copy())

              #Count large_bloom and number of bloom per period 
              for period in Q.index.values:

                  #Data per period
                  data=df[str(period)]

                  #Number of blooms per period
                  n_blooms=data.loc[data['bloom_number'].notnull(),'bloom_number'].unique()
                  Q.loc[str(period),'n_blooms']=len(n_blooms)

                  #Analysis of blooms larger than 10 days 
                  if len(n_blooms)>0:
                      total_length=0       #total lenght of blooms per period
                      n_large_blooms=0     #number of large blooms 

                      for n_bloom in n_blooms:
                          #Data per bloom
                          mask=data['bloom_number']==n_bloom
                          temp=data.loc[mask,'bloom_number']
                          bloom_lenght=len(temp)

                          #Flag for large blooms
                          if bloom_lenght>=min_bloom_length:
                              #Count large blooms
                              n_large_blooms+=1

                              #Total bloom lenght 
                              total_length=total_length+bloom_lenght

                  #Number of large blooms per period and total days of blooms per period
                  if len(n_blooms)>0:
                      if n_large_blooms==0:
                          n_large_blooms=np.NaN
                          total_length=np.NaN

                      Q.loc[str(period),'n_large_blooms']=n_large_blooms
                      Q.loc[str(period),'n_days_bloom']=total_length

              #Number of blooms
              n_large_blooms=Q['n_large_blooms'].sum()
              n_large_bloom_periods=Q['n_large_blooms'].count()

              #Ratio of periods with bloom(s) ratio
              bloom_periods_ratio=n_large_bloom_periods/n_periods

              #print('min-bloom-length {} days with max-gaps {} days: \
  #large-blooms, periods-with-large-bloom(s), and bloom-periods-ratio are {:0.0f}, {}, and {:0.2f}' \
                    #.format(min_bloom_length,max_gaps_days,n_large_blooms,n_large_bloom_periods,bloom_periods_ratio))

              #[9] Save KB period output 
              Q.to_csv('Output/KB_period_{}{}L{}G{}_{}_{}_{}.csv'.format(Period,end_of_year,min_bloom_length,max_gaps_days,start_year,end_of_year,Zone))

              #[10] Plot KB blooms
              if Plot_KB_Flag==1:
                  plot_KB(data_plot_flag,df,Q,min_bloom_length,max_gaps_days,n_large_blooms,n_large_bloom_periods,bloom_periods_ratio,start_year,end_of_year,Zone)

      
  #display(res)
  return df,Q

In [90]:
Plot_KB_Flag=0  #[1] Plot KB raw data and bloom analysis 
Period='Q'      #Q: 3 month period, 2Q: 6 month period
Zone=3          #Zone 1: Tampa, Zone 2: Apalachee Bay, Zone 3: Panhandle
start_year=1950
end_of_year=2020
df,Q=ProcessData(Plot_KB_Flag,Period,start_year,end_of_year,Zone)

Analysis from 1950 to 2020 with rule2 0.2 and bloom lenght [10] day and max gaps [5] days for period Q in Zone 3


In [96]:
start_year=1950
end_of_year=2020
columns=['n_samples_Tampa','n_samples_b1e5_Tampa',\
        'n_samples_Apalachee_Bay','n_samples_b1e5_Apalachee_Bay',\
         'n_samples_Panhandle','n_samples_b1e5_Panhandle']

for Zone in range(1,4):

  #Read data
  df_ns=pd.read_csv('Output/KB_daily_{}_{}_Zone{}.csv'.format(start_year,end_of_year,Zone))
  df_ns['date'] = pd.to_datetime(df_ns['date'])
  df_ns.index=df_ns['date']

  #Select data and resample yearly
  df_ns=df_ns[['num_of_samples','num_of_samples_b1e5']]
  df_ns=df_ns.resample('Y',kind='period').sum()

  #Create new dataframe to collect data of different zones
  if Zone==1:
    df2= pd.DataFrame(columns=columns, index=df_ns.index)
    cont=0
  df2.iloc[:,cont]=df_ns['num_of_samples']
  df2.iloc[:,cont+1]=df_ns['num_of_samples_b1e5']
  cont=cont+2
df2=df2.fillna(0)
df2.to_csv('Output/Samples_{}_{}.csv'.format(start_year,end_of_year))
display(df2.head(5))

,n_samples_Tampa,n_samples_b1e5_Tampa,n_samples_Apalachee_Bay,n_samples_b1e5_Apalachee_Bay,n_samples_Panhandle,n_samples_b1e5_Panhandle
date,,,,,,
1953,15.0,14.0,0.0,0.0,0.0,0.0
1954,408.0,157.0,0.0,0.0,0.0,0.0
1955,1177.0,0.0,0.0,0.0,0.0,0.0
1956,1290.0,0.0,0.0,0.0,0.0,0.0
1957,2164.0,884.0,0.0,0.0,0.0,0.0
